In [1]:
import warnings
warnings.filterwarnings("ignore")

from typing import Optional, List, Union
from numpy.typing import NDArray

import numpy as np
import pandas as pd

from tsururu.strategies import StrategiesFactory
from tsururu.dataset import TSDataset

In [2]:
# for global and multivariate all time seires should be in one .csv file, divided by id 
# for local each time serie should be in its own .csv file also with id column 
df_path = "datasets/global/simulated_data_to_check.csv"

# Configure the features settings
columns_and_features_params = {
    "target": {
        "column": ["y"],
        "type": "continious",
        
        "drop_raw_feature": False,
                },
    
    "date": {
        "column": ["ds"],
        "type": "datetime",
        
        "drop_raw_feature": False,
    },
    "id": {
        "column": ["unique_id"],
        "type": "categorical",
        "drop_raw_feature": False,
    }
}

# Configure the model parameters
model_params = {
    "h": 12,
    "input_size": 24,
    "hidden_size": 8,
    "conv_hidden_size": 8,
    "learning_rate": 1e-3,
    "max_steps": 10,
    "freq": "D"       # date frequency str aka 'D', 'M' or int 7, 30
}

# Configure the validation parameters
validation_params = None

# Configure the strategies parameters
strategy_params = {
    "is_multivariate": True,  
    "strategy_name": "MIMOStrategy",
    "strategy_params": {
        "horizon": 12,
        "model_name": "AutoformerRegressor_NN",
        "model_params": model_params,
        "validation_params": validation_params,
    }
}

In [3]:
strategies_factory = StrategiesFactory()

In [4]:
df=pd.read_csv(df_path,sep=',')

In [5]:
dataset = TSDataset(
    data=df,
    columns_and_features_params=columns_and_features_params,
    history=12
)

strategy = strategies_factory[strategy_params]

In [6]:
ids, test, pred, fit_time, forecast_time, num_iterations = strategy.back_test(dataset, cv=1)

Seed set to 1


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 31.47it/s]


In [7]:
def get_results(
    cv: int,
    regime: str,
    y_true: Optional[List[NDArray[np.floating]]] = None,
    y_pred: Optional[List[NDArray[np.floating]]] = None,
    ids: Optional[List[Union[float, str]]] = None,
) -> pd.DataFrame:
    def _get_fold_value(
        value: Optional[Union[float, NDArray[np.floating]]], idx: int
    ) -> List[Optional[Union[float, NDArray[np.floating]]]]:
        if value is None:
            return [None]
        if isinstance(value[idx], float):
            return value[idx]
        if isinstance(value[idx], np.ndarray):
            return value[idx].reshape(-1)
        raise TypeError(f"Unexpected value type. Value: {value}")

    df_res_dict = {}

    for idx_fold in range(cv):
        # Fill df_res_dict
        for name, value in [("y_true", y_true), ("y_pred", y_pred)]:
            df_res_dict[f"{name}_{idx_fold+1}"] = _get_fold_value(
                value, idx_fold
            )
        if regime != "local":
            df_res_dict[f"id_{idx_fold+1}"] = _get_fold_value(ids, idx_fold)

    # Save datasets to specified directory
    df_res = pd.DataFrame(df_res_dict)
    return df_res

In [8]:
get_results(cv=1, regime="local", y_true=test, y_pred=pred, ids=ids)

,y_true_1,y_pred_1
0,1988.0,1974.964844
1,1989.0,1975.761719
2,1990.0,1975.788818
3,1991.0,1975.814209
4,1992.0,1975.837158
...,...,...
115,10995.0,10975.967773
116,10996.0,10976.074219
117,10997.0,10976.197266
118,10998.0,10975.919922
